In [ ]:
#project website: https://www.kaggle.com/competitions/plant-seedlings-classification/overview

In [12]:
import numpy as np
import pandas as pd
import tensorflow as tf
import cv2
import matplotlib.pyplot as plt
import seaborn as sns
from PIL import Image
import os
import keras
from tensorflow.keras import Sequential
from tensorflow.keras.layers import Conv2D, MaxPool2D, Dropout, Flatten, Dense, BatchNormalization
from tensorflow.keras.preprocessing.image import ImageDataGenerator

In [13]:
def read_images_into_rgb_array(directory, label_val):
    import cv2
    image_files = [file for file in os.listdir(directory) if file.endswith(('.jpg', '.jpeg', '.png', '.gif'))]

    images_array2 = []
    label = []

    for file_name in image_files:
        file_path = os.path.join(directory, file_name)
        try:
            with Image.open(file_path) as img:
                
                # fixing directory path 
                str_file_pth = file_path.split("\\")
                file_path =  str_file_pth[0]+"/"+str_file_pth[1]                            
                #print("file path: {}".format(file_path))
                
                #read image
                image = cv2.imread(file_path)
                #print(image.shape)
                # convert to image to array
                img_array = Image.fromarray(image,'RGB')
                #image sizes are not fix, in order not to have 
                # problem at CNN input, we need to have fix
                # size of image size. Therefore, we resize 
                # the input
                resize_img = img_array.resize((60,60))
                images_array2.append(np.array(resize_img))
                label.append(label_val)
                
        except Exception as e:
            print(f"Error reading {file_path}: {e}")

    return ( images_array2, label )

In [14]:
import os

train_folder = "./train"

# Get the list of subdirectories in the train folder
subdirectories = [d for d in os.listdir(train_folder) if os.path.isdir(os.path.join(train_folder, d))]

# Print the list of subdirectories
for i, subdir in enumerate(subdirectories):
    print(f"Index: {i}, Subdirectory: {subdir}")

Index: 0, Subdirectory: Black-grass
Index: 1, Subdirectory: Charlock
Index: 2, Subdirectory: Cleavers
Index: 3, Subdirectory: Common Chickweed
Index: 4, Subdirectory: Common wheat
Index: 5, Subdirectory: Fat Hen
Index: 6, Subdirectory: Loose Silky-bent
Index: 7, Subdirectory: Maize
Index: 8, Subdirectory: Scentless Mayweed
Index: 9, Subdirectory: Shepherds Purse
Index: 10, Subdirectory: Small-flowered Cranesbill
Index: 11, Subdirectory: Sugar beet


In [24]:
#X = [];
#Y = [];

X = np.zeros((0, 60, 60, 3))  # Assuming your images are of size 50x50x3
Y = np.zeros((0,))  # Assuming your labels are 1D

for i, subdir in enumerate(subdirectories):
    print(f"Index: {i}, Subdirectory: {subdir}")
    [ img, label ] = read_images_into_rgb_array("./train/"+subdir, i)
    
    if len(X)==0 :
        X = img;
        Y = label;
        
    else: 
        X = np.concatenate((X, img), axis=0)
        Y = np.concatenate((Y, label), axis=0)
    

Index: 0, Subdirectory: Black-grass
<class 'list'>
Index: 1, Subdirectory: Charlock
<class 'list'>
Index: 2, Subdirectory: Cleavers
<class 'list'>
Index: 3, Subdirectory: Common Chickweed
<class 'list'>
Index: 4, Subdirectory: Common wheat
<class 'list'>
Index: 5, Subdirectory: Fat Hen
<class 'list'>
Index: 6, Subdirectory: Loose Silky-bent
<class 'list'>
Index: 7, Subdirectory: Maize
<class 'list'>
Index: 8, Subdirectory: Scentless Mayweed
<class 'list'>
Index: 9, Subdirectory: Shepherds Purse
<class 'list'>
Index: 10, Subdirectory: Small-flowered Cranesbill
<class 'list'>
Index: 11, Subdirectory: Sugar beet
<class 'list'>


In [26]:
X.shape, Y.shape

((4750, 60, 60, 3), (4750,))

In [27]:
#convert list to array for further processing
# infected
X = np.array(X)
Y = np.array(Y)

In [28]:
s=np.arange(X.shape[0])
np.random.shuffle(s)
X=X[s]
Y=Y[s]

In [29]:
#normalization 
X = X/255;
Y = Y;

In [30]:
num_classes=len(np.unique(Y))
len_data=len(X)

In [32]:
(x_train,x_test)=X[(int)(0.1*len_data):],X[:(int)(0.1*len_data)]
(y_train,y_test)=Y[(int)(0.1*len_data):],Y[:(int)(0.1*len_data)]

In [33]:
y_train=keras.utils.to_categorical(y_train,num_classes)
y_test=keras.utils.to_categorical(y_test,num_classes)

In [35]:
import numpy as np
import tensorflow as tf
from tensorflow.keras import layers, models

# Define the CNN model
model = models.Sequential()

# Convolutional layers
# input layer size is equal to frame pixel size
model.add(layers.Conv2D(32, (4, 4), activation='relu', input_shape=(60, 60, 3)))
model.add(layers.MaxPooling2D((2, 2)))
model.add(layers.Conv2D(256, (4, 4), activation='relu'))
model.add(layers.MaxPooling2D((2, 2)))
model.add(layers.Conv2D(200, (4, 4), activation='relu'))
model.add(layers.MaxPooling2D((2, 2)))

# Flatten the output for fully connected layers
model.add(layers.Flatten())

# Fully connected layers
model.add(layers.Dense(264, activation='relu'))
model.add(layers.Dense(12, activation='softmax')) 

In [36]:
model.compile(loss='binary_crossentropy', optimizer='adam', metrics=['accuracy'])

In [37]:
history = model.fit(x_train, y_train, batch_size=20, epochs=30, validation_split=0.1)

Epoch 1/30
193/193 [==============================] - 14s 67ms/step - loss: 0.2765 - accuracy: 0.2173 - val_loss: 0.2375 - val_accuracy: 0.3294
Epoch 2/30
193/193 [==============================] - 13s 68ms/step - loss: 0.2076 - accuracy: 0.4211 - val_loss: 0.1789 - val_accuracy: 0.5421
Epoch 3/30
193/193 [==============================] - 14s 71ms/step - loss: 0.1638 - accuracy: 0.5711 - val_loss: 0.1560 - val_accuracy: 0.6028
Epoch 4/30
193/193 [==============================] - 14s 73ms/step - loss: 0.1338 - accuracy: 0.6805 - val_loss: 0.1225 - val_accuracy: 0.7173
Epoch 5/30
193/193 [==============================] - 15s 76ms/step - loss: 0.1139 - accuracy: 0.7320 - val_loss: 0.1047 - val_accuracy: 0.7547
Epoch 6/30
193/193 [==============================] - 14s 73ms/step - loss: 0.0989 - accuracy: 0.7726 - val_loss: 0.0950 - val_accuracy: 0.7921
Epoch 7/30
193/193 [==============================] - 15s 76ms/step - loss: 0.0881 - accuracy: 0.7993 - val_loss: 0.0962 - val_accuracy:

In [38]:
test_loss, test_accuracy = model.evaluate(x_test, y_test, verbose=2)
print(f"Test Accuracy: {test_accuracy}")

15/15 - 0s - loss: 0.1685 - accuracy: 0.8295 - 418ms/epoch - 28ms/step
Test Accuracy: 0.8294736742973328
